In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

INFO 05-19 19:22:00 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model_path = "Qwen/Qwen3-32B"
llm = LLM(model_path, max_model_len=32768)
tokenizer = AutoTokenizer.from_pretrained(model_path)

INFO 05-19 19:22:36 [config.py:689] This model supports multiple tasks: {'reward', 'embed', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 05-19 19:22:36 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-19 19:22:39 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='Qwen/Qwen3-32B', speculative_config=None, tokenizer='Qwen/Qwen3-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_m

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


INFO 05-19 19:22:58 [loader.py:458] Loading weights took 11.10 seconds
INFO 05-19 19:22:58 [gpu_model_runner.py:1291] Model loading took 61.0347 GiB and 16.520868 seconds
INFO 05-19 19:23:15 [backends.py:416] Using cache directory: /nfs/home/tanz/.cache/vllm/torch_compile_cache/30c2679ab4/rank_0_0 for vLLM's torch.compile
INFO 05-19 19:23:15 [backends.py:426] Dynamo bytecode transform time: 17.46 s
INFO 05-19 19:23:17 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 05-19 19:23:43 [monitor.py:33] torch.compile takes 17.46 s in total
INFO 05-19 19:23:46 [kv_cache_utils.py:634] GPU KV cache size: 66,992 tokens
INFO 05-19 19:23:46 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 2.04x
INFO 05-19 19:24:12 [gpu_model_runner.py:1626] Graph capturing finished in 26 secs, took 1.21 GiB
INFO 05-19 19:24:12 [core.py:163] init engine (profile, create kv cache, warmup model) took 73.83 seconds
INFO 05-19 19:24:12 [core_client.py:435] C

# Zero-shot

In [14]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": """## Multi-Label Analysis of "Local" Identity Definition Standards in Social Network Texts

### 1. Task Objective

Analyze the given social media text segment to identify the prominent standards that the **author themself** explicitly expresses agreement with, or **implicitly agrees with** through the subtext, for defining a **"local person" (or its synonyms)**. Based on the Recognition Logic (RL) framework in this manual, output all corresponding RL category labels. A text segment may correspond to **one or more** RL categories.

**Core Limitation:** This task **only focuses** on how the author defines "local person" identity. If the text merely discusses wealth, social status, quality of life, property advantages/disadvantages, job quality, etc., **without explicitly or strongly implying** that these are standards used to judge whether someone is a "local person," then **do not annotate**.

### 2. Core Principles
*   **Focus on the Author's Core Argument:** Your judgment **must** be based on the core viewpoint, claim, or evaluation regarding the definition of "local person" identity as presented by the **speaker (i.e., the text author)**. Understanding **what the author intends to say** is the primary task.
*   **Analyze the Functional Role of Recognition Logic:** The identified recognition logic elements are not just isolated features; understand the **functional role** they play in the author's construction of their core argument.
*   **Actively Infer Context and Implicit Logic:** Actively perform contextual reasoning to understand the basis of legitimacy for "local person" identity or the evaluation criteria endorsed by the author behind their discourse.
*   **Exclude Resisted Standards and Irrelevant Discussions:** For standards that the author explicitly expresses resistance to, denies, or merely describes as being used against them by others, **do not annotate**. For discussions not directly related to defining "local person" identity, **do not annotate**.

### 3. Recognition Logic Framework

**Uniform Sentence Template:**
The author believes that people (or the author themself) should/often/can use the logic or standards represented by [Recognition Logic Type] to define who is/is not a 'local person', or to evaluate the quality/scope of a 'local' area.

**Recognition Logic Type Definitions and Features:**

#### Recognition Logic 1 (RL1): Vernacular Spatial Authority
*   **Definition:** Shared, habitual, or historically sedimented local perceptions of intra-city spatial categories that the author endorses or describes. These perceptions are not based on official administrative boundaries but reflect collective emotional maps, used as cultural shorthand for assigning evaluative or symbolic labels.
*   **Core Identifying Features:**
    *   The author explicitly proposes or assumes a logic for classifying **which areas "count" or "do not count" as core local areas, or which areas have identity or cognitive differences** (e.g., "Outside the Third Ring Road is not considered Chengdu").
    *   The author implies the status of a specific area within the "local" identity system by describing its **symbolic meaning, historical labels, or common societal views (collective emotional map)**.
    *   The author discusses **the social recognition changes or current consensus on concepts like "urban area scope," "city boundaries," etc.**

#### Recognition Logic 2 (RL2): Administrative Legitimacy
*   **Definition:** Appeals to official jurisdiction, legal status, or administrative designation. Speakers in this category justify inclusion or exclusion based on hukou registration (a household registration system), district incorporation, or municipal redistricting.
*   **Core Identifying Features:** Keywords include "administrative division," "incorporated into," "belongs to," "where is the hukou from," "ID card prefix," etc., used as a basis for judging whether a person/place is administratively/legally "local."

#### Recognition Logic 3 (RL3): Family Rootedness (Family Historical Roots / Individual Growth History)
*   **Definition:** The author endorses or cites the evaluation of local legitimacy based on the generational depth of family settlement. Claims in this category emphasize lineage, ancestry, or long-term familial ties to the area.
*   **Core Identifying Features:**
    *   Emphasizes terms like "**born and raised locally**," "**generations**," "**ancestors**," "**parents' generation**," "**within three generations**," "**came since childhood/kindergarten**," etc., to prove someone is a "genuine local" or has formed the basis of a "local person's" identity.
    *   Describes identity differences due to migration history (or lack thereof).

#### Recognition Logic 4 (RL4): Linguistic-Cultural Recognition
*   **Definition:** Relies on dialect, accent, or cultural linguistic habits as a boundary marker. Regional speech features are treated as proxies for insider status, and deviations often provoke mockery or mistrust. May also include identification with specific local cultural habits (e.g., customs, lifestyle).
*   **Core Identifying Features:** Mentions "speaking the local dialect," "accent," "cannot understand/stand certain accents," "don't you understand our local rules," etc., as criteria for distinguishing insiders from outsiders, or judging if someone is "one of us" or possesses "local attributes."

#### Recognition Logic 5 (RL5): Functional Livability (Convenience of Living Functions and Environmental Quality Perception)
*   **Definition:** The author endorses or cites the evaluation of urban areas in terms of their material infrastructure (e.g., transit, housing, education, or access to services), often to assert spatial superiority or desirability.
*   **Core Identifying Features:**
    *   Mentions "subway," "amenities," "convenience," "education," "good greening," "few people," "streetscape," "comfortable," etc., and **directly associates these with an evaluation of whether an area is "good," "livable," or "worth living in."**
    *   The author considers an area an ideal "local" living space due to its RL5 characteristics, or believes an area does not meet the standard of a "good local" area due to a lack of RL5 characteristics.

#### Recognition Logic 6 (RL6): Social Embeddedness (Depth of Social Roots and Symbolism of Economic Status)
*   **Definition:** The author endorses or cites judging whether someone is a local based on their integration into local social circles and possession of local fixed assets. This includes community resources (like dividends, familiarity with the community) and material or symbolic resources, such as (inherited) property.
*   **Core Identifying Features:**
    *   Mentions "connections (renmai)," "dividends," "old relationships," "community influence," "friends nearby," "has several properties in the city center," "impact of high/low housing prices on identity," "spending one or two million to buy a house," etc., and **explicitly or implicitly links these to the "stability, authenticity, hierarchy of local identity, or evaluation criteria for a region."**

#### Recognition Logic 7 (RL7): Occupational Typification
*   **Definition:** The author endorses or cites framing identity by associating certain districts with dominant professional groups, such as civil servants, migrant workers, or business owners, thereby invoking implicit hierarchies of class and worth.
*   **Core Identifying Features:** Mentions specific occupations or types of people (e.g., "farmers," "those who farm," "migrant workers"), and **strongly associates them with the "local attributes," resident composition, or social stratification of a specific area**, thereby defining identity or evaluating the area.

### 4. Annotation Procedure
1.  **Step 1: Identify named entities mentioned in the text**
    *   Read through the text and record the entities mentioned.
2.  **Step 2: Match arguments to each named entity**
    *   Read the text carefully and match the corresponding viewpoint statements to each mentioned entity.
3.  **Step 3: Analyze the underlying recognition logic behind each viewpoint statement**
    *   For each entity, analyze which recognition logic(s) the speaker's expressed viewpoint (if any) is based on. It can be multiple recognition logics.
4.  **Step 4: Output the results**
    *   Output all matched **one or more** recognition logic category labels [Recognition Logic N], separated by ``, `` (a comma followed by a space).

### 5. Important Notes
*   Constantly remind yourself that the annotation target is **standards that the author themself does not explicitly oppose and uses to define "local person" identity or evaluate a "local" area, and understand their function in the author's core argument.**
*   Do not rely solely on keyword matching; deeply understand the logic, intent, and core viewpoint behind the author's discourse. **Follow the steps from Step 1 to Step 4 in 4. Annotation Procedure; do not skip steps.**
*   If the author is merely describing a phenomenon, expressing personal preference, or evaluating social status/wealth, without revealing that they use these standards to define "who is a local person" or evaluate "the quality/scope/distinction of a local area," then **do not annotate.**
*   Output the result after the `Output: ` marker. **Stop immediately** after outputting the result; do not output any further content."""
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [15]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 500
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "**Step 1: Understand the Author's Core Expressive Intent**\n "
        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [16]:
import re
import pandas as pd

# --- Define English Output Markers ---
# Primary regex for "Output:" (as specified in the prompt)
# It looks for "Output:" followed by optional whitespace, and captures the rest.
primary_output_marker_pattern_en = re.compile(r"Output:\s*(.*)", re.DOTALL)

# Secondary regex for the step header "Step 6: Output Result"
# This is a fallback if the primary marker isn't found.
secondary_output_marker_pattern_en = re.compile(r"Step 6: Output Result\s*(.*)", re.DOTALL)

# Tertiary regex for the step header "**Step 6: Output**" (with markdown bold)
# This is another fallback.
tertiary_output_marker_pattern_en = re.compile(r"\*\*Step 6: Output\*\*\s*(.*)", re.DOTALL)

# Regex for splitting by the simple "Output:" marker - used as the last fallback
simple_output_marker_regex_en = r"输出："
# --- End of English Markers ---


# Assuming all_outputs is defined from a previous cell, containing the model's full outputs
# Assuming texts is defined from a previous cell, containing the original input texts
# Assuming model_path is defined from a previous cell, used for naming the output file

outputs = all_outputs
texts = texts # Ensure texts is accessible if it's not global

parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]
    # Assuming the structure of output_obj remains consistent (e.g., from vertexai response)
    # Access the text content, handling potential structure variations if necessary
    model_full_output_text = ""
    try:
        # Common ways to access text from different model APIs
        if hasattr(output_obj, 'text'): # e.g., some older simple text responses
             model_full_output_text = output_obj.text
        elif hasattr(output_obj, 'content'): # e.g., some newer structures
             model_full_output_text = output_obj.content
        elif hasattr(output_obj, 'outputs') and output_obj.outputs and hasattr(output_obj.outputs[0], 'text'): # e.g., vertexai list of outputs
             model_full_output_text = output_obj.outputs[0].text
        else:
             # Fallback or raise error if text cannot be found
             print(f"Warning: Could not find text content in output_obj structure for index {i}. Output object: {output_obj}")
             model_full_output_text = str(output_obj) # Use string representation as a last resort
             # Consider adding a specific error type or logging here
    except Exception as e:
         print(f"Error accessing text content for index {i}: {e}")
         model_full_output_text = str(output_obj) # Use string representation on error


    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary English output marker first ("Output:")
    match_primary = primary_output_marker_pattern_en.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (Output:)"
    else:
        # If primary marker is not found, try the secondary English marker (step-based "Step 6: Output Result")
        match_secondary = secondary_output_marker_pattern_en.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (Step 6: Output Result)"
        else:
            # If secondary marker is not found, try the tertiary English marker (step-based "**Step 6: Output**")
            match_tertiary = tertiary_output_marker_pattern_en.search(model_full_output_text)
            if match_tertiary:
                extracted_label_part = match_tertiary.group(1).strip()
                used_marker_type = "tertiary (**Step 6: Output**)"
            else:
                # If tertiary marker is not found, try the simple plain "输出：" marker.
                # We split by the marker and take the content after the *last* occurrence.
                parts = re.split(simple_output_marker_regex_en, model_full_output_text)
                if len(parts) > 1: # Marker was found
                    extracted_label_part = parts[-1].strip()
                    used_marker_type = f"simple (plain '{simple_output_marker_regex_en}' marker)"

    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()

        # Check if the label is empty after stripping wrappers or is the explicit "N/A" output
        if not label or label == "N/A":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "N/A"
                 print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                 print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label

    else:
        # Log a warning if none of the English output markers are found
        print(f"Warning: Could not find any of the expected English markers: primary ('{primary_output_marker_pattern_en.pattern}'), secondary ('{secondary_output_marker_pattern_en.pattern}'), tertiary ('{tertiary_output_marker_pattern_en.pattern}'), or simple plain ('{simple_output_marker_regex_en}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename - you might want to add something like "_en"
# Assuming model_path is defined in a previous cell.
output_csv_filename = f"results/{model_path.split('/')[-1]}_zero_shot_en.csv" # Added _en

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Successfully processed 483 model outputs.
Results saved to Qwen3-32B_zero_shot_en.csv

First 5 rows of the parsed data:
                                 Original_Input_Text   RL_Types  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那... 

# Few-shot

In [17]:
def generate_prompt(input_text: str):
    return [
        {
      "role": "system",
      "content": """## Multi-Label Analysis of "Local" Identity Definition Standards in Social Network Texts

### 1. Task Objective

Analyze the given social media text segment to identify the prominent standards that the **author themself** explicitly expresses agreement with, or **implicitly agrees with** through the subtext, for defining a **"local person" (or its synonyms)**. Based on the Recognition Logic (RL) framework in this manual, output all corresponding RL category labels. A text segment may correspond to **one or more** RL categories.

**Core Limitation:** This task **only focuses** on how the author defines "local person" identity. If the text merely discusses wealth, social status, quality of life, property advantages/disadvantages, job quality, etc., **without explicitly or strongly implying** that these are standards used to judge whether someone is a "local person," then **do not annotate**.

### 2. Core Principles
*   **Focus on the Author's Core Argument:** Your judgment **must** be based on the core viewpoint, claim, or evaluation regarding the definition of "local person" identity as presented by the **speaker (i.e., the text author)**. Understanding **what the author intends to say** is the primary task.
*   **Analyze the Functional Role of Recognition Logic:** The identified recognition logic elements are not just isolated features; understand the **functional role** they play in the author's construction of their core argument.
*   **Actively Infer Context and Implicit Logic:** Actively perform contextual reasoning to understand the basis of legitimacy for "local person" identity or the evaluation criteria endorsed by the author behind their discourse.
*   **Exclude Resisted Standards and Irrelevant Discussions:** For standards that the author explicitly expresses resistance to, denies, or merely describes as being used against them by others, **do not annotate**. For discussions not directly related to defining "local person" identity, **do not annotate**.

### 3. Recognition Logic Framework

**Uniform Sentence Template:**
The author believes that people (or the author themself) should/often/can use the logic or standards represented by [Recognition Logic Type] to define who is/is not a 'local person', or to evaluate the quality/scope of a 'local' area.

**Recognition Logic Type Definitions and Features:**

#### Recognition Logic 1 (RL1): Vernacular Spatial Authority
*   **Definition:** Shared, habitual, or historically sedimented local perceptions of intra-city spatial categories that the author endorses or describes. These perceptions are not based on official administrative boundaries but reflect collective emotional maps, used as cultural shorthand for assigning evaluative or symbolic labels.
*   **Core Identifying Features:**
    *   The author explicitly proposes or assumes a logic for classifying **which areas "count" or "do not count" as core local areas, or which areas have identity or cognitive differences** (e.g., "Outside the Third Ring Road is not considered Chengdu").
    *   The author implies the status of a specific area within the "local" identity system by describing its **symbolic meaning, historical labels, or common societal views (collective emotional map)**.
    *   The author discusses **the social recognition changes or current consensus on concepts like "urban area scope," "city boundaries," etc.**

#### Recognition Logic 2 (RL2): Administrative Legitimacy
*   **Definition:** Appeals to official jurisdiction, legal status, or administrative designation. Speakers in this category justify inclusion or exclusion based on hukou registration (a household registration system), district incorporation, or municipal redistricting.
*   **Core Identifying Features:** Keywords include "administrative division," "incorporated into," "belongs to," "where is the hukou from," "ID card prefix," etc., used as a basis for judging whether a person/place is administratively/legally "local."

#### Recognition Logic 3 (RL3): Family Rootedness (Family Historical Roots / Individual Growth History)
*   **Definition:** The author endorses or cites the evaluation of local legitimacy based on the generational depth of family settlement. Claims in this category emphasize lineage, ancestry, or long-term familial ties to the area.
*   **Core Identifying Features:**
    *   Emphasizes terms like "**born and raised locally**," "**generations**," "**ancestors**," "**parents' generation**," "**within three generations**," "**came since childhood/kindergarten**," etc., to prove someone is a "genuine local" or has formed the basis of a "local person's" identity.
    *   Describes identity differences due to migration history (or lack thereof).

#### Recognition Logic 4 (RL4): Linguistic-Cultural Recognition
*   **Definition:** Relies on dialect, accent, or cultural linguistic habits as a boundary marker. Regional speech features are treated as proxies for insider status, and deviations often provoke mockery or mistrust. May also include identification with specific local cultural habits (e.g., customs, lifestyle).
*   **Core Identifying Features:** Mentions "speaking the local dialect," "accent," "cannot understand/stand certain accents," "don't you understand our local rules," etc., as criteria for distinguishing insiders from outsiders, or judging if someone is "one of us" or possesses "local attributes."

#### Recognition Logic 5 (RL5): Functional Livability (Convenience of Living Functions and Environmental Quality Perception)
*   **Definition:** The author endorses or cites the evaluation of urban areas in terms of their material infrastructure (e.g., transit, housing, education, or access to services), often to assert spatial superiority or desirability.
*   **Core Identifying Features:**
    *   Mentions "subway," "amenities," "convenience," "education," "good greening," "few people," "streetscape," "comfortable," etc., and **directly associates these with an evaluation of whether an area is "good," "livable," or "worth living in."**
    *   The author considers an area an ideal "local" living space due to its RL5 characteristics, or believes an area does not meet the standard of a "good local" area due to a lack of RL5 characteristics.

#### Recognition Logic 6 (RL6): Social Embeddedness (Depth of Social Roots and Symbolism of Economic Status)
*   **Definition:** The author endorses or cites judging whether someone is a local based on their integration into local social circles and possession of local fixed assets. This includes community resources (like dividends, familiarity with the community) and material or symbolic resources, such as (inherited) property.
*   **Core Identifying Features:**
    *   Mentions "connections (renmai)," "dividends," "old relationships," "community influence," "friends nearby," "has several properties in the city center," "impact of high/low housing prices on identity," "spending one or two million to buy a house," etc., and **explicitly or implicitly links these to the "stability, authenticity, hierarchy of local identity, or evaluation criteria for a region."**

#### Recognition Logic 7 (RL7): Occupational Typification
*   **Definition:** The author endorses or cites framing identity by associating certain districts with dominant professional groups, such as civil servants, migrant workers, or business owners, thereby invoking implicit hierarchies of class and worth.
*   **Core Identifying Features:** Mentions specific occupations or types of people (e.g., "farmers," "those who farm," "migrant workers"), and **strongly associates them with the "local attributes," resident composition, or social stratification of a specific area**, thereby defining identity or evaluating the area.

### 4. Annotation Procedure
1.  **Step 1: Identify named entities mentioned in the text**
    *   Read through the text and record the entities mentioned.
2.  **Step 2: Match arguments to each named entity**
    *   Read the text carefully and match the corresponding viewpoint statements to each mentioned entity.
3.  **Step 3: Analyze the underlying recognition logic behind each viewpoint statement**
    *   For each entity, analyze which recognition logic(s) the speaker's expressed viewpoint (if any) is based on. It can be multiple recognition logics.
4.  **Step 4: Output the results**
    *   Output all matched **one or more** recognition logic category labels [Recognition Logic N], separated by ``, `` (a comma followed by a space).

### 5. Important Notes
*   Constantly remind yourself that the annotation target is **standards that the author themself does not explicitly oppose and uses to define "local person" identity or evaluate a "local" area, and understand their function in the author's core argument.**
*   Do not rely solely on keyword matching; deeply understand the logic, intent, and core viewpoint behind the author's discourse. **Follow the steps from Step 1 to Step 4 in 4. Annotation Procedure; do not skip steps.**
*   If the author is merely describing a phenomenon, expressing personal preference, or evaluating social status/wealth, without revealing that they use these standards to define "who is a local person" or evaluate "the quality/scope/distinction of a local area," then **do not annotate.**
*   Output the result after the `Output: ` marker. **Stop immediately** after outputting the result; do not output any further content."""
      },
    { # FEW-SHOT EXAMPLE 1 (RL1)
      "role": "user",
      "content": "Well, people from Chongzhou, when asked where they're from, they directly say Chongzhou, not Chengdu. What I've learned since I was young is that Chengdu people generally think areas outside the Third Ring Road don't count as Chengdu."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author wants to express that they and people around them, when identifying their "local" identity, will directly say "Chongzhou" and not "Chengdu"; furthermore, the author has been influenced by a long-standing social convention since childhood that Chengdu people generally believe areas outside the city's Third Ring Road are not considered part of the core "Chengdu" local area. This reflects a standard for defining "local people" based on geographical scope and social custom.

2. Identify All Potential RL Elements:
- People around the author identify their area as "Chongzhou", not "Chengdu", reflecting regional cognition and identity definition.
- "Learned since I was young" (ear濡目染) indicates this cognition is a long-standing social habit.
- "Areas outside the Third Ring Road don't count as Chengdu" clearly demarcates a geographical range, defining which areas are considered the local core.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
These geographical boundary definitions, along with the identification habits of the author and those around them, collectively form a set of socially conventional regional cognition standards used to define who is local, thereby confirming the local identity difference between "Chongzhou" and "Chengdu".

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author endorses and implicitly expresses the use of long-standing socially conventional regional cognition (such as the Third Ring Road division and associated city identity) to define "local people" identity.

5. Precisely Match RL Categories and Construct an Explanation:
This text aligns with RL1 because the author emphasizes using long-standing social conventional cognition and regional scope demarcation to judge and express the belonging and identity of "local people".

Output: RL1"""
    },
    { # FEW-SHOT EXAMPLE 2 (RL2)
      "role": "user",
      "content": "Hugs, anyway, in my eyes, I only look at administrative divisions. If it belongs to Qingdao, then they are Qingdao people, regardless of whether it's Jiaozhou or Chengyang. In earlier years, because transportation was inconvenient, people felt they were far apart, that's all. Now, whether it's by subway or car, you can get there in a moment."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author explicitly states that they believe "local people" identity should be based on administrative divisions. That is, as long as an area belongs to the administrative region of Qingdao (including Jiaozhou, Chengyang), its residents are considered Qingdao people. This is presented as the standard for judging local identity. The author also mentions that improved transportation convenience has changed past perceptions of distance.

2. Identify All Potential RL Elements:
- "I only look at administrative divisions," "If it belongs to Qingdao, then they are Qingdao people," highlights administrative belonging as the defining standard.
- Specific districts (Jiaozhou, Chengyang) are mentioned, indicating that members within these administrative boundaries are considered local.
- "Transportation was inconvenient," "subway," "car, you can get there in a moment," although describing functional livability, serve as background to explain how perceptions related to administrative importance have changed, not as a standard for local identity itself.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
Administrative division is the fundamental standard the author uses to define "local people" identity. Transportation convenience is supplementary information that supports the shift in perception regarding local identity based on administrative boundaries.

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author significantly endorses using administrative divisions as the standard for judging "local people."

5. Precisely Match RL Categories and Construct an Explanation:
This text reflects a definition of local identity based on administrative legitimacy and belonging, which aligns with RL2.

Output: RL2"""
    },
    { # FEW-SHOT EXAMPLE 3 (RL3)
      "role": "user",
      "content": "In Guangzhou, within three generations counts as a true Guangzhou person, within two generations is a new Guangzhou person, one generation of pioneers are Guangzhou immigrants, and those whose families haven't spoken Cantonese for three generations are Guangzhou people from other provinces (ancestors were 'lao佬' - outsiders) who came to Guangzhou."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author clearly expresses that family lineage (intergenerational heritage) is the standard for defining "local people" identity. They use different generational markers to classify identities such as "true Guangzhou person," "new Guangzhou person," "Guangzhou immigrant," and "Guangzhou people from other provinces," emphasizing the importance of family roots.

2. Identify All Potential RL Elements:
- "Within three generations," "within two generations," "one generation of pioneers" are clearly definitions based on family historical rootedness and growth history.
- "Haven't spoken Cantonese for three generations" implies cultural-linguistic recognition.
- However, the core thrust of the argument is based on generational lines, emphasizing lineage, heritage, and intergenerational identity.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
Family historical rootedness is the primary standard the author uses to strictly define Guangzhou local identity. Generational heritage determines "true" belonging. Although language is mentioned, it is more to describe cultural differences of specific generations rather than being the primary standard.

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author endorses and explicitly expresses family historical rootedness (family generational heritage) as the core standard for defining local identity.

5. Precisely Match RL Categories and Construct an Explanation:
This text primarily reflects a logic of local identity based on family historical rootedness, aligning with RL3.

Output: RL3"""
    },
    { # FEW-SHOT EXAMPLE 4 (RL4)
      "role": "user",
      "content": "I'm a Chengdu native, I think it's okay! To be honest, I just really can't stand some accents."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author identifies as a "Chengdu native," indicating their local identity. Simultaneously, mentioning they "can't stand some accents" implies that language accent serves as a standard or boundary for distinguishing "local people" from "non-local people."

2. Identify All Potential RL Elements:
- "I'm a Chengdu native" indicates an identity based on being born or having long-term residence in the area.
- "Can't stand some accents" pertains to cultural-linguistic recognition.
- Accent acts as a distinguishing marker between locals and non-locals.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
The author, through their self-identified "native" status and sensitivity to accents, demonstrates cultural-linguistic recognition as a standard for judging locals versus non-locals, implying an endorsement or exclusion based on cultural-linguistic traits.

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author implicitly endorses cultural-linguistic differences, such as accent, as a standard for defining local identity.

5. Precisely Match RL Categories and Construct an Explanation:
This text aligns with the RL4 category, which is based on using language and cultural characteristics to identify local people.

Output: RL4"""
    },
    { # FEW-SHOT EXAMPLE 5 (RL5)
      "role": "user",
      "content": "That's why I like Beichen, it's a transition from the old city to the new district. The streetscape isn't as dilapidated as the old city's, it's only a dozen minutes from the city center, has all the amenities, and it's so comfortable to live there."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author expresses fondness for the Beichen area, primarily due to its living convenience and environmental quality, such as street conditions, proximity to the city center, and available amenities, stating it's "so comfortable to live there." This reflects the author's criteria for evaluating the quality of different local areas.

2. Identify All Potential RL Elements:
- "The streetscape isn't as dilapidated as the old city's" highlights differences in environmental quality.
- "Only a dozen minutes from the city center" emphasizes transportation convenience.
- "Has all the amenities" refers to functional livability.
- "Comfortable to live there" reflects an overall positive assessment of the area's quality of life.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
These descriptions support the author's positive evaluation of Beichen as a high-quality local area. This implies that Beichen offers ideal local living conditions, showcasing the author's use of living convenience and environmental quality as standards for assessing the quality of local areas and their "local value."

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author explicitly uses environmental quality and convenience of amenities as standards for measuring and choosing an ideal local residential area.

5. Precisely Match RL Categories and Construct an Explanation:
The text aligns with the RL5 standard, based on functional livability and environmental quality cognition, i.e., using living comfort and the quality of amenities to evaluate the hierarchy and "local value" of areas within the locality.

Output: RL5"""
    },
    { # FEW-SHOT EXAMPLE 6 (RL6)
      "role": "user",
      "content": "After all, not many families can afford one or two million to buy a house in a better environment and still within the second ring road. My parents have worked in Chengdu for so long, mostly within the first or second ring road, and friends are also nearby. Buying in the suburbs is definitely unrealistic; buying and renovating a house within the first or second ring road is the best solution."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author expresses how economic capacity and social networks influence the choice of property location. They believe that owning property within the second ring road, close to work and friends, is a more realistic and ideal living choice. This implicitly suggests that property location and economic status serve as indicators of "local" identity and criteria for evaluating regional desirability.

2. Identify All Potential RL Elements:
- "Buy a house in a better environment and still within the second ring road" emphasizes property location and quality.
- "My parents have worked in Chengdu for so long," "friends are also nearby" reflect social connections and roots in the community.
- "Buying in the suburbs is definitely unrealistic" indicates a distinction between "good local areas" and "peripheral areas."
- "Buying and renovating a house within the first or second ring road is the best solution" reflects a comprehensive consideration of economic conditions and living environment.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
The author implicitly links economic strength, existing social ties (distribution of relatives and friends), and ownership of property in a core area as important standards for judging the stability of local identity and the value of a region. This expresses an endorsement of property within the second ring road and the social rootedness it implies.

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author implicitly endorses owning property in a core area (like within the second ring road) and having stable social connections as important standards for local identity and regional quality.

5. Precisely Match RL Categories and Construct an Explanation:
This text primarily reflects the influence of social rootedness and economic status symbolism on the determination of local identity, aligning with RL6.

Output: RL6"""
    },
    { # FEW-SHOT EXAMPLE 7 (RL7)
      "role": "user",
      "content": "Licang and Huangdao are rural areas, with many farmers. The older generation doesn't consider them city people."
    },
    {
      "role": "assistant",
      "content": """1. Understand the Author's Core Expressive Intent:
The author mentions "farmers" as an occupational identity and implies that the older generation excludes farmers from their definition of "city people." This indicates that occupational identity plays a role in defining local identity.

2. Identify All Potential RL Elements:
- "Farmers" as a specific occupational identity.
- "The older generation doesn't consider them city people" implies identity exclusion or definition based on occupation.

3. Analyze the Functional Role of RL Elements in the Author's Core Argument:
Occupational identity is actually used as a standard to distinguish "city people" from non-city people, carrying symbolic meaning.

4. Filter RL Criteria Related to the Core Argument and Endorsed by the Author:
The author (by relaying the older generation's view without refuting it in this context of defining local distinctions) acknowledges occupational identity (farmer) as a criterion for distinguishing between locals and non-locals, or different social groups within the local context.

5. Precisely Match RL Categories and Construct an Explanation:
Using "farmer" as an occupational identity to define local status aligns with the occupational symbolism standard, thus qualifying as RL7.

Output: RL7"""
    },
    {
      "role": "user",
      "content": input_text
    },
  ]

In [7]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 500
# sampling_params = SamplingParams(temperature=0.6, top_p=0.95, top_k=20, min_p=0, max_tokens=8192)
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        ) + "1. Understand the Author's Core Expressive Intent:"

        formatted_prompts_batch.append(formatted_prompt)
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [8]:
import re
import pandas as pd

# --- Define English Output Markers ---
# Primary regex for "Output:" (as specified in the prompt)
# It looks for "Output:" followed by optional whitespace, and captures the rest.
primary_output_marker_pattern_en = re.compile(r"Output:\s*(.*)", re.DOTALL)

# Secondary regex for the step header "Step 6: Output Result"
# This is a fallback if the primary marker isn't found.
secondary_output_marker_pattern_en = re.compile(r"Step 6: Output Result\s*(.*)", re.DOTALL)

# Tertiary regex for the step header "**Step 6: Output**" (with markdown bold)
# This is another fallback.
tertiary_output_marker_pattern_en = re.compile(r"\*\*Step 6: Output\*\*\s*(.*)", re.DOTALL)

# Regex for splitting by the simple "Output:" marker - used as the last fallback
simple_output_marker_regex_en = r"输出："
# --- End of English Markers ---


# Assuming all_outputs is defined from a previous cell, containing the model's full outputs
# Assuming texts is defined from a previous cell, containing the original input texts
# Assuming model_path is defined from a previous cell, used for naming the output file

outputs = all_outputs
texts = texts # Ensure texts is accessible if it's not global

parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]
    # Assuming the structure of output_obj remains consistent (e.g., from vertexai response)
    # Access the text content, handling potential structure variations if necessary
    model_full_output_text = ""
    try:
        # Common ways to access text from different model APIs
        if hasattr(output_obj, 'text'): # e.g., some older simple text responses
             model_full_output_text = output_obj.text
        elif hasattr(output_obj, 'content'): # e.g., some newer structures
             model_full_output_text = output_obj.content
        elif hasattr(output_obj, 'outputs') and output_obj.outputs and hasattr(output_obj.outputs[0], 'text'): # e.g., vertexai list of outputs
             model_full_output_text = output_obj.outputs[0].text
        else:
             # Fallback or raise error if text cannot be found
             print(f"Warning: Could not find text content in output_obj structure for index {i}. Output object: {output_obj}")
             model_full_output_text = str(output_obj) # Use string representation as a last resort
             # Consider adding a specific error type or logging here
    except Exception as e:
         print(f"Error accessing text content for index {i}: {e}")
         model_full_output_text = str(output_obj) # Use string representation on error


    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary English output marker first ("Output:")
    match_primary = primary_output_marker_pattern_en.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (Output:)"
    else:
        # If primary marker is not found, try the secondary English marker (step-based "Step 6: Output Result")
        match_secondary = secondary_output_marker_pattern_en.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (Step 6: Output Result)"
        else:
            # If secondary marker is not found, try the tertiary English marker (step-based "**Step 6: Output**")
            match_tertiary = tertiary_output_marker_pattern_en.search(model_full_output_text)
            if match_tertiary:
                extracted_label_part = match_tertiary.group(1).strip()
                used_marker_type = "tertiary (**Step 6: Output**)"
            else:
                # If tertiary marker is not found, try the simple plain "输出：" marker.
                # We split by the marker and take the content after the *last* occurrence.
                parts = re.split(simple_output_marker_regex_en, model_full_output_text)
                if len(parts) > 1: # Marker was found
                    extracted_label_part = parts[-1].strip()
                    used_marker_type = f"simple (plain '{simple_output_marker_regex_en}' marker)"

    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()

        # Check if the label is empty after stripping wrappers or is the explicit "N/A" output
        if not label or label == "N/A":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "N/A"
                 print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                 print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label

    else:
        # Log a warning if none of the English output markers are found
        print(f"Warning: Could not find any of the expected English markers: primary ('{primary_output_marker_pattern_en.pattern}'), secondary ('{secondary_output_marker_pattern_en.pattern}'), tertiary ('{tertiary_output_marker_pattern_en.pattern}'), or simple plain ('{simple_output_marker_regex_en}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename - you might want to add something like "_en"
# Assuming model_path is defined in a previous cell.
output_csv_filename = f"results/{model_path.split('/')[-1]}_few_shot_en.csv" # Added _en

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):

Problematic Raw Output (after marker):



In [9]:
results_df['RL_Types']

0                  RL1
1      RL1, RL4, RL3
2                  RL4
3            RL1, RL4
4                  RL4
             ...       
478          RL1, RL3
479                RL3
480                RL1
481                RL2
482          RL3, RL6
Name: RL_Types, Length: 483, dtype: object

# No CoT

In [10]:
def generate_prompt(input_text: str):
    return [
    {
      "role": "system",
      "content": """## Multi-Label Analysis of "Local" Identity Definition Standards in Social Network Texts

### 1. Task Objective

Analyze the given social media text segment to identify the prominent standards that the **author themself** explicitly expresses agreement with, or **implicitly agrees with** through the subtext, for defining a **"local person" (or its synonyms)**. Based on the Recognition Logic (RL) framework in this manual, output all corresponding RL category labels. A text segment may correspond to **one or more** RL categories.

**Core Limitation:** This task **only focuses** on how the author defines "local person" identity. If the text merely discusses wealth, social status, quality of life, property advantages/disadvantages, job quality, etc., **without explicitly or strongly implying** that these are standards used to judge whether someone is a "local person," then **do not annotate**.

### 2. Core Principles
*   **Focus on the Author's Core Argument:** Your judgment **must** be based on the core viewpoint, claim, or evaluation regarding the definition of "local person" identity as presented by the **speaker (i.e., the text author)**. Understanding **what the author intends to say** is the primary task.
*   **Analyze the Functional Role of Recognition Logic:** The identified recognition logic elements are not just isolated features; understand the **functional role** they play in the author's construction of their core argument.
*   **Actively Infer Context and Implicit Logic:** Actively perform contextual reasoning to understand the basis of legitimacy for "local person" identity or the evaluation criteria endorsed by the author behind their discourse.
*   **Exclude Resisted Standards and Irrelevant Discussions:** For standards that the author explicitly expresses resistance to, denies, or merely describes as being used against them by others, **do not annotate**. For discussions not directly related to defining "local person" identity, **do not annotate**.

### 3. Recognition Logic Framework

**Uniform Sentence Template:**
The author believes that people (or the author themself) should/often/can use the logic or standards represented by [Recognition Logic Type] to define who is/is not a 'local person', or to evaluate the quality/scope of a 'local' area.

**Recognition Logic Type Definitions and Features:**

#### Recognition Logic 1 (RL1): Vernacular Spatial Authority
*   **Definition:** Shared, habitual, or historically sedimented local perceptions of intra-city spatial categories that the author endorses or describes. These perceptions are not based on official administrative boundaries but reflect collective emotional maps, used as cultural shorthand for assigning evaluative or symbolic labels.
*   **Core Identifying Features:**
    *   The author explicitly proposes or assumes a logic for classifying **which areas "count" or "do not count" as core local areas, or which areas have identity or cognitive differences** (e.g., "Outside the Third Ring Road is not considered Chengdu").
    *   The author implies the status of a specific area within the "local" identity system by describing its **symbolic meaning, historical labels, or common societal views (collective emotional map)**.
    *   The author discusses **the social recognition changes or current consensus on concepts like "urban area scope," "city boundaries," etc.**

#### Recognition Logic 2 (RL2): Administrative Legitimacy
*   **Definition:** Appeals to official jurisdiction, legal status, or administrative designation. Speakers in this category justify inclusion or exclusion based on hukou registration (a household registration system), district incorporation, or municipal redistricting.
*   **Core Identifying Features:** Keywords include "administrative division," "incorporated into," "belongs to," "where is the hukou from," "ID card prefix," etc., used as a basis for judging whether a person/place is administratively/legally "local."

#### Recognition Logic 3 (RL3): Family Rootedness (Family Historical Roots / Individual Growth History)
*   **Definition:** The author endorses or cites the evaluation of local legitimacy based on the generational depth of family settlement. Claims in this category emphasize lineage, ancestry, or long-term familial ties to the area.
*   **Core Identifying Features:**
    *   Emphasizes terms like "**born and raised locally**," "**generations**," "**ancestors**," "**parents' generation**," "**within three generations**," "**came since childhood/kindergarten**," etc., to prove someone is a "genuine local" or has formed the basis of a "local person's" identity.
    *   Describes identity differences due to migration history (or lack thereof).

#### Recognition Logic 4 (RL4): Linguistic-Cultural Recognition
*   **Definition:** Relies on dialect, accent, or cultural linguistic habits as a boundary marker. Regional speech features are treated as proxies for insider status, and deviations often provoke mockery or mistrust. May also include identification with specific local cultural habits (e.g., customs, lifestyle).
*   **Core Identifying Features:** Mentions "speaking the local dialect," "accent," "cannot understand/stand certain accents," "don't you understand our local rules," etc., as criteria for distinguishing insiders from outsiders, or judging if someone is "one of us" or possesses "local attributes."

#### Recognition Logic 5 (RL5): Functional Livability (Convenience of Living Functions and Environmental Quality Perception)
*   **Definition:** The author endorses or cites the evaluation of urban areas in terms of their material infrastructure (e.g., transit, housing, education, or access to services), often to assert spatial superiority or desirability.
*   **Core Identifying Features:**
    *   Mentions "subway," "amenities," "convenience," "education," "good greening," "few people," "streetscape," "comfortable," etc., and **directly associates these with an evaluation of whether an area is "good," "livable," or "worth living in."**
    *   The author considers an area an ideal "local" living space due to its RL5 characteristics, or believes an area does not meet the standard of a "good local" area due to a lack of RL5 characteristics.

#### Recognition Logic 6 (RL6): Social Embeddedness (Depth of Social Roots and Symbolism of Economic Status)
*   **Definition:** The author endorses or cites judging whether someone is a local based on their integration into local social circles and possession of local fixed assets. This includes community resources (like dividends, familiarity with the community) and material or symbolic resources, such as (inherited) property.
*   **Core Identifying Features:**
    *   Mentions "connections (renmai)," "dividends," "old relationships," "community influence," "friends nearby," "has several properties in the city center," "impact of high/low housing prices on identity," "spending one or two million to buy a house," etc., and **explicitly or implicitly links these to the "stability, authenticity, hierarchy of local identity, or evaluation criteria for a region."**

#### Recognition Logic 7 (RL7): Occupational Typification
*   **Definition:** The author endorses or cites framing identity by associating certain districts with dominant professional groups, such as civil servants, migrant workers, or business owners, thereby invoking implicit hierarchies of class and worth.
*   **Core Identifying Features:** Mentions specific occupations or types of people (e.g., "farmers," "those who farm," "migrant workers"), and **strongly associates them with the "local attributes," resident composition, or social stratification of a specific area**, thereby defining identity or evaluating the area.

### 4. Important Notes
*   Constantly remind yourself that the annotation target is **standards that the author themself does not explicitly oppose and uses to define "local person" identity or evaluate a "local" area, and understand their function in the author's core argument.**
*   If the author is merely describing a phenomenon, expressing personal preference, or evaluating social status/wealth, without revealing that they use these standards to define "who is a local person" or evaluate "the quality/scope/distinction of a local area," then **do not annotate.**
*   Output the result after the `Output: ` marker. **Stop immediately** after outputting the result; do not output any further content."""
    },
    {
      "role": "user",
      "content": input_text
    }
  ]

In [11]:
import pandas as pd
df = pd.read_csv('sample.csv')
texts = df['text'].tolist()

all_outputs = []
batch_size = 500
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=8192)

for i in range(0, len(texts), batch_size):
    current_batch_texts = texts[i:i + batch_size]
    
    if not current_batch_texts: # Should not happen if texts is not empty, but good practice
        continue

    formatted_prompts_batch = []
    for p_text in current_batch_texts:
        prompt_data = generate_prompt(p_text)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_data,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False  # Setting enable_thinking=False disables thinking mode
        )
        formatted_prompts_batch.append(formatted_prompt + "Output: ")
    
    # Generate outputs for the current batch
    # The llm.generate call might show a progress bar if it's integrated with tqdm,
    # which would reset for each batch. This is acceptable.
    batch_outputs = llm.generate(formatted_prompts_batch, sampling_params)
    all_outputs.extend(batch_outputs)

outputs = all_outputs # The final list of all generated outputs

Processed prompts:   0%| | 0/483 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [12]:
import re
import pandas as pd

# --- Define English Output Markers ---
# Primary regex for "Output:" (as specified in the prompt)
# It looks for "Output:" followed by optional whitespace, and captures the rest.
primary_output_marker_pattern_en = re.compile(r"Output:\s*(.*)", re.DOTALL)

# Secondary regex for the step header "Step 6: Output Result"
# This is a fallback if the primary marker isn't found.
secondary_output_marker_pattern_en = re.compile(r"Step 6: Output Result\s*(.*)", re.DOTALL)

# Tertiary regex for the step header "**Step 6: Output**" (with markdown bold)
# This is another fallback.
tertiary_output_marker_pattern_en = re.compile(r"\*\*Step 6: Output\*\*\s*(.*)", re.DOTALL)

# Regex for splitting by the simple "Output:" marker - used as the last fallback
simple_output_marker_regex_en = r"输出："
# --- End of English Markers ---


# Assuming all_outputs is defined from a previous cell, containing the model's full outputs
# Assuming texts is defined from a previous cell, containing the original input texts
# Assuming model_path is defined from a previous cell, used for naming the output file

outputs = all_outputs
texts = texts # Ensure texts is accessible if it's not global

parsed_results_list = []

for i, output_obj in enumerate(outputs):
    original_text_from_dataset = texts[i]
    # Assuming the structure of output_obj remains consistent (e.g., from vertexai response)
    # Access the text content, handling potential structure variations if necessary
    model_full_output_text = ""
    try:
        # Common ways to access text from different model APIs
        if hasattr(output_obj, 'text'): # e.g., some older simple text responses
             model_full_output_text = output_obj.text
        elif hasattr(output_obj, 'content'): # e.g., some newer structures
             model_full_output_text = output_obj.content
        elif hasattr(output_obj, 'outputs') and output_obj.outputs and hasattr(output_obj.outputs[0], 'text'): # e.g., vertexai list of outputs
             model_full_output_text = output_obj.outputs[0].text
        else:
             # Fallback or raise error if text cannot be found
             print(f"Warning: Could not find text content in output_obj structure for index {i}. Output object: {output_obj}")
             model_full_output_text = str(output_obj) # Use string representation as a last resort
             # Consider adding a specific error type or logging here
    except Exception as e:
         print(f"Error accessing text content for index {i}: {e}")
         model_full_output_text = str(output_obj) # Use string representation on error

    model_full_output_text = "Output: " + model_full_output_text
    # Initialize a dictionary to store data for the current item
    current_parsed_item = {
        "Original_Input_Text": original_text_from_dataset,
        "RL_Types": "PARSE_ERROR_NO_MARKER", # Default if marker not found or parsing fails
        "Raw_Model_Output": model_full_output_text # Store the raw model output for debugging
    }

    extracted_label_part = None
    used_marker_type = None # To log which marker was successful

    # Try to find the primary English output marker first ("Output:")
    match_primary = primary_output_marker_pattern_en.search(model_full_output_text)
    if match_primary:
        extracted_label_part = match_primary.group(1).strip()
        used_marker_type = "primary (Output:)"
    else:
        # If primary marker is not found, try the secondary English marker (step-based "Step 6: Output Result")
        match_secondary = secondary_output_marker_pattern_en.search(model_full_output_text)
        if match_secondary:
            extracted_label_part = match_secondary.group(1).strip()
            used_marker_type = "secondary (Step 6: Output Result)"
        else:
            # If secondary marker is not found, try the tertiary English marker (step-based "**Step 6: Output**")
            match_tertiary = tertiary_output_marker_pattern_en.search(model_full_output_text)
            if match_tertiary:
                extracted_label_part = match_tertiary.group(1).strip()
                used_marker_type = "tertiary (**Step 6: Output**)"
            else:
                # If tertiary marker is not found, try the simple plain "输出：" marker.
                # We split by the marker and take the content after the *last* occurrence.
                parts = re.split(simple_output_marker_regex_en, model_full_output_text)
                if len(parts) > 1: # Marker was found
                    extracted_label_part = parts[-1].strip()
                    used_marker_type = f"simple (plain '{simple_output_marker_regex_en}' marker)"

    if extracted_label_part is not None:
        label = ""
        # Attempt to clean the extracted part to get the pure label
        # Case 1: Markdown code block like ```\nLABEL\n```
        if extracted_label_part.startswith("```\n") and extracted_label_part.endswith("\n```"):
            label = extracted_label_part[len("```\n") : -len("\n```")].strip()
        # Case 2: Backticks like `LABEL`
        elif extracted_label_part.startswith("`") and extracted_label_part.endswith("`"):
            label = extracted_label_part[1:-1].strip()
        # Case 3: No special formatting, just the text (it might be the label itself)
        else:
            label = extracted_label_part.strip()

        # Check if the label is empty after stripping wrappers or is the explicit "N/A" output
        if not label or label == "N/A":
            current_parsed_item["RL_Types"] = "N/A" # Consistent representation for no labels
            if not label: # Log if it was empty due to stripping, not if it was explicitly "N/A"
                 print(f"Warning: Extracted label was empty after stripping wrappers for original text at index {i} (using {used_marker_type} marker).")
                 print(f"Problematic Raw Output (after marker):\n{extracted_label_part}\n" + "="*30)
        else:
            current_parsed_item["RL_Types"] = label

    else:
        # Log a warning if none of the English output markers are found
        print(f"Warning: Could not find any of the expected English markers: primary ('{primary_output_marker_pattern_en.pattern}'), secondary ('{secondary_output_marker_pattern_en.pattern}'), tertiary ('{tertiary_output_marker_pattern_en.pattern}'), or simple plain ('{simple_output_marker_regex_en}') in model output for original text at index {i}.")
        print(f"Problematic Raw Output:\n{model_full_output_text}\n" + "="*30)
        # RL_Types remains "PARSE_ERROR_NO_MARKER" as set during initialization

    parsed_results_list.append(current_parsed_item)

# Create a Pandas DataFrame from the list of parsed dictionaries
results_df = pd.DataFrame(parsed_results_list)

# Define the output CSV filename - you might want to add something like "_en"
# Assuming model_path is defined in a previous cell.
output_csv_filename = f"results/{model_path.split('/')[-1]}_no_cot_en.csv" # Added _en

# Save the DataFrame to a CSV file
results_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')

print(f"Successfully processed {len(parsed_results_list)} model outputs.")
print(f"Results saved to {output_csv_filename}")

# Display the first few rows of the DataFrame as a quick check
print("\nFirst 5 rows of the parsed data:")
print(results_df.head())

Successfully processed 483 model outputs.
Results saved to Qwen3-32B_no_cot_en.csv

First 5 rows of the parsed data:
                                 Original_Input_Text   RL_Types  \
0  热烈欢迎咱叙北第一带盐人 北门可不差，论三环外，东西三环可不一定有北三环好，再等火北弄好，那...        RL1   
1  真的很多，我就是周边的，不管现实还是网上看的很多，他们天天说自己土著然后说我们弯脚杆，碰到过...       1,4,7   
2  我是都江堰的，其他地方的我不敢说，但是我可是从小被成都口音嘲笑哦，我小时候去亲戚家耍过暑假，...  RL1, RL4   
3  那我说都江堰话被笑的更多，我以前被介绍了个成都人，然后天天给我说他家以前二环以内的，说他家以...  RL1, RL4   
4  那证明成都人的确不咋地，本来成都话口音离普通话语区就够偏了，很多怪音，他们还歧视本省的人，无...  RL1, RL4   

      Raw_Model_Output  
0        Output:  RL1  
1        Output: 1,4,7  
2  Output:  RL1, RL4  
3  Output:  RL1, RL4  
4  Output:  RL1, RL4  


In [13]:
results_df['RL_Types']

0            RL1
1           1,4,7
2      RL1, RL4
3      RL1, RL4
4      RL1, RL4
          ...    
478             1
479          RL3
480          RL1
481          RL1
482          RL1
Name: RL_Types, Length: 483, dtype: object